In [1]:
# @zhangjoy
# @多因子策略
# @20180330

In [2]:
# 因子选择：th主题热度因子 + 
# [tmv总市值 + cmv流通市值 + PB市净率 + PE市盈率 + RSI相对强弱指标 + WMS威廉指数 + tor换手率 tinysoft] + 
# [ROA资产利润率 + ROE净资产收益率 wind]

In [3]:
import pandas as pd
import heapq

In [4]:
# 读取数据，对比 tushare 与 tinysoft wind 的 hs300成分股差异
# 统一以 tushare 为标准，修改股票名称
th_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/lda/data/stock_topicfactor_all.csv")


cmv_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/cmv.csv",encoding="gbk")
tmv_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/tmv.csv",encoding="gbk")
pb_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/pb.csv",encoding="gbk")
pe_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/pe.csv",encoding="gbk")
wms_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/wms.csv",encoding="gbk")
tor_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/tor.csv",encoding="gbk")
rsi_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/rsi.csv",encoding="gbk")


roa_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/roa.csv",encoding="gbk")
roe_df = pd.read_csv("/Users/zhangjoy/Documents/vscode/py_project/multi_factor_strategy/data/roe.csv",encoding="gbk")

In [5]:
'''
# 统一不同数据库取股票数据的股票名称不一致性
stock_name1 = th_df['name'].tolist()
stock_name2 = rsi_df['股票名称'].tolist()
for i in stock_name1:
    if i not in stock_name2:
        print(i)
print()
for i in stock_name2:
    if i not in stock_name1:
        print(i)
# 所有股票的收盘价数据都爬取下来了！
for i in stock_name1:
    path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + i + ".csv"
    try:
        s_df = pd.read_csv(path)
    except:
        print(i)

# index_o = th_df.name.tolist().index('石化油服') 
# th_df.code.tolist()[index_o] # 600871.csv  将 石化油服 改为 *ST油服
'''

'\n# 统一不同数据库取股票数据的股票名称不一致性\nstock_name1 = th_df[\'name\'].tolist()\nstock_name2 = rsi_df[\'股票名称\'].tolist()\nfor i in stock_name1:\n    if i not in stock_name2:\n        print(i)\nprint()\nfor i in stock_name2:\n    if i not in stock_name1:\n        print(i)\n# 所有股票的收盘价数据都爬取下来了！\nfor i in stock_name1:\n    path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + i + ".csv"\n    try:\n        s_df = pd.read_csv(path)\n    except:\n        print(i)\n\n# index_o = th_df.name.tolist().index(\'石化油服\') \n# th_df.code.tolist()[index_o] # 600871.csv  将 石化油服 改为 *ST油服\n'

In [5]:
# 获取模型自变量的值（每月要训练1次模型，然后做模型预测选股）
# 选择 name 和因子值
th = th_df.ix[:,2:]
# cmv_df、tmv_df、pb_df、pe_df、wms_df、tor_df、rsi_df、roa_df、roe_df

# 每月建模数据准备
# 月初建仓 月末平仓
buy_cmv = ['股票名称','2016/4/1','2016/5/3','2016/6/1','2016/7/1','2016/8/1','2016/9/1','2016/10/10','2016/11/1','2016/12/1','2017/1/3','2017/2/3','2017/3/1','2017/4/5','2017/5/2','2017/6/1','2017/7/3','2017/8/1','2017/9/1']
buy_else = ['股票名称','2016-04-01','2016-05-03','2016-06-01','2016-07-01','2016-08-01','2016-09-01','2016-10-10','2016-11-01','2016-12-01','2017-01-03','2017-02-03','2017-03-01','2017-04-05','2017-05-02','2017-06-01','2017-07-03','2017-08-01','2017-09-01']

cmv = cmv_df[buy_cmv]
tmv = tmv_df[buy_else]
pb = pb_df[buy_else]
pe = pe_df[buy_else]
wms = wms_df[buy_else]
tor = tor_df[buy_else]
rsi = rsi_df[buy_else]
# 列重命名，新的列名['201604','201605','201606','201607','201608','201609','201610','201611','201612','201701','201702','201703','201704','201705','201706','201707','201708','201709']
cmv.columns = th.columns.values.tolist()
tmv.columns = th.columns.values.tolist()
pb.columns = th.columns.values.tolist()
pe.columns = th.columns.values.tolist()
wms.columns = th.columns.values.tolist()
tor.columns = th.columns.values.tolist()
rsi.columns = th.columns.values.tolist()

roa = roa_df
roe = roe_df


# 按股票名称排序
# th = th.sort_values(by='name',axis=0,ascending=False)
# cmv = cmv.sort_values(by='name',axis=0,ascending=False)
# tmv = tmv.sort_values(by='name',axis=0,ascending=False)
# pb = pb.sort_values(by='name',axis=0,ascending=False)
# pe = pe.sort_values(by='name',axis=0,ascending=False)
# wms = wms.sort_values(by='name',axis=0,ascending=False)
# tor = tor.sort_values(by='name',axis=0,ascending=False)
# rsi = rsi.sort_values(by='name',axis=0,ascending=False)
# roa = roa.sort_values(by='name',axis=0,ascending=False)
# roe = roe.sort_values(by='name',axis=0,ascending=False)

/Users/zhangjoy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
stock_name = th['name'].tolist() # 全部股票名称

In [7]:
# 先筛选股票，后建模
# 剔除st、pt
STPT = ['ST保千里', '*ST油服'] 

# 剔除上市不满半年的股票 从网站爬取上市时间，计算得到：18个列表【代码见 stock_publish_time,ipynb】
NoHalfyear = [
 ['华安证券','江苏银行','杭州银行','中国电影','新城控股','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','招商蛇口','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','新城控股','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','招商蛇口','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','新城控股','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','招商蛇口','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','第一创业','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','中国核建','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','浙商证券','中国银河','玲珑轮胎','贵阳银行','汇顶科技','欧派家居','步长制药','裕同科技','张家港行','视源股份'],
 ['华安证券','江苏银行','杭州银行','中国电影','三角轮胎','白银有色','广州港','上海银行','中原证券','浙商证券','中国银河','贵阳银行','汇顶科技','欧派家居','步长制药','裕同科技','张家港行','视源股份'],
 ['华安证券','杭州银行','三角轮胎','白银有色','广州港','上海银行','中原证券','浙商证券','中国银河','汇顶科技','欧派家居','步长制药','裕同科技','张家港行','视源股份'],
 ['华安证券','杭州银行','白银有色','广州港','上海银行','中原证券','浙商证券','中国银河','汇顶科技','欧派家居','步长制药','裕同科技','张家港行','视源股份'],
 ['华安证券','白银有色','广州港','上海银行','中原证券','浙商证券','中国银河','欧派家居','步长制药','裕同科技','张家港行','视源股份'],
 ['华安证券','白银有色','广州港','中原证券','浙商证券','中国银河','欧派家居','裕同科技','张家港行','视源股份'],
 ['白银有色', '广州港', '中原证券', '浙商证券', '中国银河', '欧派家居', '张家港行', '视源股份'],
 ['白银有色', '广州港', '浙商证券', '欧派家居'],
 ['广州港', '浙商证券', '欧派家居']]

# 剔除一个投资期有3个交易日以上未交易的股票 18个列表
NoThreedays = []
s_name = '华夏银行'
path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
s_df = pd.read_csv(path).sort_values(by='datetime',axis=0, ascending=True) # 按日期升序排序
tradetime = s_df['datetime'].tolist() # 交易日数据
tradetime_list =[tradetime[0:20],tradetime[20:41],tradetime[41:61],tradetime[61:82],tradetime[82:105],tradetime[105:125],tradetime[125:141],tradetime[141:163],tradetime[163:185],tradetime[185:203],tradetime[203:221],tradetime[221:244],tradetime[244:262],tradetime[262:282],tradetime[282:304],tradetime[304:325],tradetime[325:348],tradetime[348:369]]
for month in range(18):
    Nothreeday = []
    for s in range(300):
        s_name = stock_name[s]
        path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
        s_df = pd.read_csv(path)  # 读取文件数据
        num = 0
        for tradeday in tradetime_list[month]:
            try:
                index_o = s_df.datetime.tolist().index(tradeday) # 月初 2016-04-01 的索引
                s_close = s_df[index_o:index_o+1]['close'].values[0] # 月末收盘价，收盘价卖出
            except:
                num = num + 1
        if num >= 3:
            Nothreeday.append(s_name)
    NoThreedays.append(Nothreeday)
# NoThreedays

# 剔除建仓日、平仓日未交易的股票[月初、月末]：每月交易日开始日和结束日 18个列表
NoFirstEnd = []
tradetime_start = ['2016-04-01','2016-05-03','2016-06-01','2016-07-01','2016-08-01','2016-09-01','2016-10-10','2016-11-01','2016-12-01','2017-01-03','2017-02-03','2017-03-01','2017-04-05','2017-05-02','2017-06-01','2017-07-03','2017-08-01','2017-09-01']
tradetime_end = ['2016-04-29','2016-05-31','2016-06-30','2016-07-29','2016-08-31','2016-09-30','2016-10-31','2016-11-30','2016-12-30','2017-01-26','2017-02-28','2017-03-31','2017-04-28','2017-05-31','2017-06-30','2017-07-31','2017-08-31','2017-09-29']
for month in range(18):
    Nofirstend = []
    for s in range(300):
        s_name = stock_name[s]
        path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
        try:
            s_df = pd.read_csv(path)  # 读取文件数据
            index_o = s_df.datetime.tolist().index(tradetime_start[month]) # 月初 2016-04-01 的索引
            index_c = s_df.datetime.tolist().index(tradetime_end[month]) # 月末 2016-04-29 的索引
            s_open = s_df[index_o:index_o+1]['close'].values[0] # 月初收盘价, 收盘价买入
            s_close = s_df[index_c:index_c+1]['close'].values[0] # 月末收盘价，收盘价卖出
        except:
            Nofirstend.append(s_name)
    NoFirstEnd.append(Nofirstend)
# NoFirstEnd

In [8]:
print([len(NoHalfyear[i]) for i in range(18)])
print([len(NoFirstEnd[i]) for i in range(18)])
print([len(NoThreedays[i]) for i in range(18)])

[23, 23, 23, 21, 21, 21, 21, 21, 20, 19, 18, 15, 14, 12, 10, 8, 4, 3]
[46, 41, 34, 35, 35, 39, 37, 29, 30, 18, 19, 19, 16, 19, 20, 17, 20, 19]
[54, 45, 36, 36, 34, 36, 32, 31, 31, 19, 21, 21, 21, 19, 24, 18, 20, 20]


In [9]:
stock = [] # 先筛选股票
for i in range(18):
    s = th['name'].tolist() # 全部股票名称
#     print(len(s))
    for s_n in stock_name:
        if (s_n in STPT) or (s_n in NoHalfyear[i]) or (s_n in NoThreedays[i]) or (s_n in NoFirstEnd[i]):
#             print(s_n)
            s.remove(s_n)
#     print(len(s))
#     print()
    stock.append(s)

In [10]:
print([len(stock[i]) for i in range(18)])
print()

[241, 251, 258, 256, 258, 251, 253, 257, 255, 266, 263, 264, 264, 267, 265, 272, 274, 275]



In [ ]:
'''
# 输出列表最大/小的n个元素的索引
import heapq  
lst = [12, 13, 1, 8, 10]  
min_n = 2  
temp = map(lst.index, heapq.nlargest(min_n, lst))  
temp = list(temp)  
print(temp) 
'''

In [11]:
# 获取因变量的值，个股表现好的：收益率前 20%，类别为1:
stock_1 = [] # 类别为1的股票名称
tradetime_start = ['2016-04-01','2016-05-03','2016-06-01','2016-07-01','2016-08-01','2016-09-01','2016-10-10','2016-11-01','2016-12-01','2017-01-03','2017-02-03','2017-03-01','2017-04-05','2017-05-02','2017-06-01','2017-07-03','2017-08-01','2017-09-01']
tradetime_end = ['2016-04-29','2016-05-31','2016-06-30','2016-07-29','2016-08-31','2016-09-30','2016-10-31','2016-11-30','2016-12-30','2017-01-26','2017-02-28','2017-03-31','2017-04-28','2017-05-31','2017-06-30','2017-07-31','2017-08-31','2017-09-29']
for month in range(18):
    monthreturn = []
    for s in range(len(stock[month])):
        s_name = stock[month][s]
        path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
        s_df = pd.read_csv(path)
        index_o = s_df.datetime.tolist().index(tradetime_start[month]) # 月初 2016-04-01 的索引
        index_c = s_df.datetime.tolist().index(tradetime_end[month]) # 月末 2016-04-29 的索引
        s_open = s_df[index_o:index_o+1]['close'].values[0] # 月初收盘价, 收盘价买入
        s_close = s_df[index_c:index_c+1]['close'].values[0] # 月末收盘价，收盘价卖出
        mr = (s_close-s_open)/s_open # 股票月收益率
        monthreturn.append(mr)
    # 找出月收益率列表最大 20% 的索引值
    max_n = round(len(stock[month])*0.2) # 前20%有多少股票，并四舍五入
    print(max_n)
    temp = map(monthreturn.index, heapq.nlargest(max_n, monthreturn))  
    temp = list(temp) # 索引列表
    stock1 = [stock[month][i] for i in temp]
    stock_1.append(stock1)

48
50
52
51
52
50
51
51
51
53
53
53
53
53
53
54
55
55


In [13]:
# 类别标签 y
y = []
for month in range(18):
    y_s = []
    for s in stock[month]:
        if s in stock_1[month]:
            y_stock = 1
        else:
            y_stock = 0
        y_s.append(y_stock)
    y.append(y_s)

In [14]:
# 获取自变量数据：筛选股票后的自变量
# th、cmv、tmv、pb、pe、wms、tor、rsi、roa、roe
# time_list = th.columns.values.tolist()
# time_list.remove('name')
time_list = ['201604','201605','201606','201607','201608','201609','201610','201611','201612','201701','201702','201703','201704','201705','201706','201707','201708','201709']
# 获得自变量数据 + 因变量 合并
def get_logistic_data(month):
    # 自变量
#     th_stock = th['name'].tolist()
#     th_T = th.T
#     th_T.columns = th_stock
#     th_factor = th_T[stock[month]].T[time_list[month]] # pandas.core.series.Series

    cmv_stock = cmv['name'].tolist()
    cmv_T = cmv.T
    cmv_T.columns = cmv_stock
    cmv_factor = cmv_T[stock[month]].T[time_list[month]]
    # 缺失值处理：均值填充
    cmv_factor.fillna(cmv_factor.mean())  
    # 异常值处理：数值分布在（μ-3σ,μ+3σ) 中的概率为0.9974
    #lambda if 条件为真的时候返回if前面内容，否则返回0
    cmv_std = cmv_factor.std()
    cmv_mean = cmv_factor.mean()
    cmv_factor.apply(lambda x: (cmv_mean-3*cmv_std) if x<(cmv_mean-3*cmv_std) else x) # μ-3σ,μ+3σ
    cmv_factor.apply(lambda x: (cmv_mean+3*cmv_std) if x>(cmv_mean+3*cmv_std) else x) # μ-3σ,μ+3σ

    
    tmv_stock = tmv['name'].tolist()
    tmv_T = tmv.T
    tmv_T.columns = tmv_stock
    tmv_factor = tmv_T[stock[month]].T[time_list[month]]
    tmv_factor.fillna(tmv_factor.mean()) 
    tmv_std = tmv_factor.std()
    tmv_mean = tmv_factor.mean()
    tmv_factor.apply(lambda x: (tmv_mean-3*tmv_std) if x<(tmv_mean-3*tmv_std) else x) # μ-3σ,μ+3σ
    tmv_factor.apply(lambda x: (tmv_mean+3*tmv_std) if x>(tmv_mean+3*tmv_std) else x) # μ-3σ,μ+3σ


    pb_stock = pb['name'].tolist()
    pb_T = pb.T
    pb_T.columns = pb_stock
    pb_factor = pb_T[stock[month]].T[time_list[month]]
    pb_factor.fillna(pb_factor.mean()) 
    pb_std = pb_factor.std()
    pb_mean = pb_factor.mean()
    pb_factor.apply(lambda x: (pb_mean-3*pb_std) if x<(pb_mean-3*pb_std) else x) # μ-3σ,μ+3σ
    pb_factor.apply(lambda x: (pb_mean+3*pb_std) if x>(pb_mean+3*pb_std) else x) # μ-3σ,μ+3σ


    pe_stock = pe['name'].tolist()
    pe_T = pe.T
    pe_T.columns = pe_stock
    pe_factor = pe_T[stock[month]].T[time_list[month]]
    pe_factor.fillna(pe_factor.mean()) 
    pe_std = pe_factor.std()
    pe_mean = pe_factor.mean()
    pe_factor.apply(lambda x: (pe_mean-3*pe_std) if x<(pe_mean-3*pe_std) else x) # μ-3σ,μ+3σ
    pe_factor.apply(lambda x: (pe_mean+3*pe_std) if x>(pe_mean+3*pe_std) else x) # μ-3σ,μ+3σ


    wms_stock = wms['name'].tolist()
    wms_T = wms.T
    wms_T.columns = wms_stock
    wms_factor = wms_T[stock[month]].T[time_list[month]]
    wms_factor.fillna(wms_factor.mean()) 
    wms_std = wms_factor.std()
    wms_mean = wms_factor.mean()
    wms_factor.apply(lambda x: (wms_mean-3*wms_std) if x<(wms_mean-3*wms_std) else x) # μ-3σ,μ+3σ
    wms_factor.apply(lambda x: (wms_mean+3*wms_std) if x>(wms_mean+3*wms_std) else x) # μ-3σ,μ+3σ


    tor_stock = tor['name'].tolist()
    tor_T = tor.T
    tor_T.columns = tor_stock
    tor_factor = tor_T[stock[month]].T[time_list[month]]
    tor_factor.fillna(tor_factor.mean()) 
    tor_std = tor_factor.std()
    tor_mean = tor_factor.mean()
    tor_factor.apply(lambda x: (tor_mean-3*tor_std) if x<(tor_mean-3*tor_std) else x) # μ-3σ,μ+3σ
    tor_factor.apply(lambda x: (tor_mean+3*tor_std) if x>(tor_mean+3*tor_std) else x) # μ-3σ,μ+3σ


    rsi_stock = rsi['name'].tolist()
    rsi_T = rsi.T
    rsi_T.columns = rsi_stock
    rsi_factor = rsi_T[stock[month]].T[time_list[month]]
    rsi_factor.fillna(rsi_factor.mean()) 
    rsi_std = rsi_factor.std()
    rsi_mean = rsi_factor.mean()
    rsi_factor.apply(lambda x: (rsi_mean-3*rsi_std) if x<(rsi_mean-3*rsi_std) else x) # μ-3σ,μ+3σ
    rsi_factor.apply(lambda x: (rsi_mean+3*rsi_std) if x>(rsi_mean+3*rsi_std) else x) # μ-3σ,μ+3σ


    roa_stock = th['name'].tolist()
    roa_T = roa.T
    roa_T.columns = roa_stock
    roa_factor = roa_T[stock[month]].T[time_list[month]]
    roa_factor.fillna(roa_factor.mean()) 
    roa_std = roa_factor.std()
    roa_mean = roa_factor.mean()
    roa_factor.apply(lambda x: (roa_mean-3*roa_std) if x<(roa_mean-3*roa_std) else x) # μ-3σ,μ+3σ
    roa_factor.apply(lambda x: (roa_mean+3*roa_std) if x>(roa_mean+3*roa_std) else x) # μ-3σ,μ+3σ


    roe_stock = roe['name'].tolist()
    roe_T = roe.T
    roe_T.columns = roe_stock
    roe_factor = roe_T[stock[month]].T[time_list[month]]
    roe_factor.fillna(roe_factor.mean()) 
    roe_std = roe_factor.std()
    roe_mean = roe_factor.mean()
    roe_factor.apply(lambda x: (roe_mean-3*roe_std) if x<(roe_mean-3*roe_std) else x) # μ-3σ,μ+3σ
    roe_factor.apply(lambda x: (roe_mean+3*roe_std) if x>(roe_mean+3*roe_std) else x) # μ-3σ,μ+3σ

    
    # 删除原来索引，重新建立从0开始的索引
#     th_factor = th_factor.reset_index(drop=True)
    cmv_factor = cmv_factor.reset_index(drop=True)
    tmv_factor = tmv_factor.reset_index(drop=True)
    pb_factor = pb_factor.reset_index(drop=True)
    pe_factor = pe_factor.reset_index(drop=True)
    wms_factor = wms_factor.reset_index(drop=True)
    tor_factor = tor_factor.reset_index(drop=True)
    rsi_factor = rsi_factor.reset_index(drop=True)
    roa_factor = roa_factor.reset_index(drop=True)
    roe_factor = roe_factor.reset_index(drop=True)
    
#     print(th_factor)
    
    # 因变量
    y_label = pd.Series(y[month])
#     print(y_label)
    
    lst = [cmv_factor,tmv_factor,pb_factor,pe_factor,wms_factor,tor_factor,rsi_factor,roa_factor,roe_factor,y_label] #th_factor,
    data_df = pd.concat(lst,axis=1) # series 合并
    data_df.columns = ['cmv','tmv','pb','pe','wms','tor','rsi','roa','roe','y'] # 列重命名 'th',
#     print(data_df)
    return data_df # pandas.core.frame.DataFrame

In [17]:
# logistic建模
# way1
import statsmodels.api as sm
def logistic(month,allocation_weight):
    data_df = get_logistic_data(month) # pandas.core.frame.DataFrame
    print(data_df.shape)
    # 训练模型
    # 指定作为训练变量的列
    train_cols = data_df.columns[:9] # 列名X.astype(float) 9/10 是因子数量
    logit = sm.Logit(data_df['y'].astype(float), data_df[train_cols].astype(float))
    # 拟合模型
    result = logit.fit()
    # result.summary() # 训练模型结果摘要

    # 构建预测集
    # 数据中的列要跟预测时用到的列一致
    data_pre_df = data_df
    # data_pre_df = get_logistic_data(month+1)
    predict_cols = data_pre_df.columns[:9]
    # 进行预测，并将预测评分存入predict列中
    data_pre_df['predict'] = result.predict(data_pre_df[predict_cols].astype(float)) # series
    a_w = data_pre_df['predict']/data_pre_df['predict'].sum() # 归一化
    allocation_weight.append(a_w) # 配资权重


In [18]:
allocation_weight = []
for month in range(18):
    logistic(month,allocation_weight)
print(len(allocation_weight))

(241, 10)
Optimization terminated successfully.
         Current function value: 0.476689
         Iterations 7
(251, 10)
Optimization terminated successfully.
         Current function value: 0.477242
         Iterations 7
(258, 10)
Optimization terminated successfully.
         Current function value: 0.428870
         Iterations 9
(256, 10)
Optimization terminated successfully.
         Current function value: 0.451820
         Iterations 8
(258, 10)
Optimization terminated successfully.
         Current function value: 0.449421
         Iterations 8
(251, 10)
Optimization terminated successfully.
         Current function value: 0.471933
         Iterations 8
(253, 10)
Optimization terminated successfully.
         Current function value: 0.487265
         Iterations 7
(257, 10)
Optimization terminated successfully.
         Current function value: 0.443015
         Iterations 8
(255, 10)
Optimization terminated successfully.
         Current function value: 0.466470
         Itera

In [19]:
allocation_weight[0]

0      0.003912
1      0.003935
2      0.002262
3      0.001444
4      0.004091
5      0.001778
6      0.001568
7      0.003465
8      0.003736
9      0.004337
10     0.002021
11     0.000718
12     0.002454
13     0.002384
14     0.003243
15     0.003899
16     0.003879
17     0.003029
18     0.003570
19     0.003528
20     0.005910
21     0.003560
22     0.006593
23     0.005066
24     0.007410
25     0.006870
26     0.004601
27     0.004007
28     0.004135
29     0.006074
         ...   
211    0.001511
212    0.007445
213    0.004876
214    0.004427
215    0.006328
216    0.006501
217    0.002998
218    0.004737
219    0.005460
220    0.006116
221    0.008114
222    0.006187
223    0.002065
224    0.005047
225    0.006901
226    0.008169
227    0.006636
228    0.005253
229    0.006109
230    0.007427
231    0.005327
232    0.004427
233    0.009325
234    0.003827
235    0.007035
236    0.005347
237    0.004574
238    0.008493
239    0.003325
240    0.004467
Name: predict, Length: 2

In [20]:
# 辅助函数：两列表对应元素相乘并求和
def list_sum(a, b): 
    s = sum([a[i]*b[i] for i in range(len(a))])
    return s

In [21]:
# 已知：股票列表stock + 权重列表allocation_weight
# 计算每个月 月末净（资产）值 + 每个月每组股票的持股数
def asset_month(asset_list, sharenum_list):
    asset = 1
    for month in range(18):
        share_num = [] # 持股数
        end_close = [] # 月末收盘价，计算月末净值，即新的资产值
        for i in range(len(stock[month])):
            s_name = stock[month][i]
            path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
            s_df = pd.read_csv(path)  # 读取文件数据
            
            index_s = s_df.datetime.tolist().index(tradetime_start[month]) # 月初 2016-04-01 的索引
            index_e = s_df.datetime.tolist().index(tradetime_end[month]) # 月末 2016-04-29 的索引

            s_close_start = s_df[index_s:index_s+1]['close'].values[0] # 月初收盘价买入
            s_close_end = s_df[index_e:index_e+1]['close'].values[0] # 月末收盘价卖出
            
            s_n = asset * allocation_weight[month][i] / s_close_start # 持股数
            
            share_num.append(s_n)
            end_close.append(s_close_end)

        asset = list_sum(share_num,end_close) # 新的资产净值【可以考虑定额止损】
        
        asset_list.append(asset)
        sharenum_list.append(share_num)

In [22]:
asset_list, sharenum_list = [], []
asset_month(asset_list, sharenum_list)

In [23]:
asset_list

[0.9661168507556139,
 0.9255468702963244,
 0.9279704358688476,
 0.9697959598509992,
 1.031898485667576,
 1.0175313917922026,
 1.0338639905181923,
 1.0977329186306344,
 1.0315020183882682,
 1.0467313795323578,
 1.1148488964120242,
 1.1346274227687612,
 1.1059840962796075,
 1.1203620830932408,
 1.215666601947032,
 1.24529658514106,
 1.2935111497087086,
 1.3122031252267883]

In [228]:
'''
    上文已求
s_name = '华夏银行'
path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
s_df = pd.read_csv(path).sort_values(by='datetime',axis=0, ascending=True) # 按日期升序排序
tradetime = s_df['datetime'].tolist() # 交易日数据
'''

In [24]:
# 绘制净值曲线：已知 stock, sharenum_list, tradetime
tradetime_list =[tradetime[0:20],tradetime[20:41],tradetime[41:61],tradetime[61:82],tradetime[82:105],tradetime[105:125],tradetime[125:141],tradetime[141:163],tradetime[163:185],tradetime[185:203],tradetime[203:221],tradetime[221:244],tradetime[244:262],tradetime[262:282],tradetime[282:304],tradetime[304:325],tradetime[325:348],tradetime[348:369]]
def netvalue(netvalue_list):
    for month in range(18):
        for tradeday in tradetime_list[month]:
            close = [] # 收盘价列表
            for s_name in stock[month]:
                path = "/Users/zhangjoy/Documents/vscode/py_project/lda/data/stockname_data/" + s_name + ".csv"
                try:
                    s_df = pd.read_csv(path)  # 读取文件数据
                    index_c = s_df.datetime.tolist().index(tradeday) # 交易日的索引
                    s_close = s_df[index_c:index_c+1]['close'].values[0] # 交易日收盘价
                except:
                    s_close = close[-1] # 前一天收盘价填补缺失值
                close.append(s_close)
            net_value = list_sum(sharenum_list[month],close)
            netvalue_list.append(net_value)

In [25]:
netvalue_list = []
%time netvalue(netvalue_list)
netvalue_list # 3m

CPU times: user 2min 43s, sys: 8.06 s, total: 2min 51s
Wall time: 2min 53s


[0.9999999999999996,
 1.020874839108085,
 1.0233006049648816,
 1.0049907605454607,
 0.9911177673863787,
 1.0113710681611146,
 0.996890435624043,
 1.0184441922039633,
 1.0248120034292314,
 1.0217155483606695,
 1.0076145175226654,
 1.0081329697822312,
 0.9762059247670417,
 0.9717860222164625,
 0.973069958445051,
 0.9680112754562231,
 0.9760322809264917,
 0.9664153483657503,
 0.9658817743265501,
 0.9661168507556139,
 0.9661168507556139,
 0.965094004660799,
 0.9690110681312919,
 0.9382516793873357,
 0.9129865833783829,
 0.9138830523207847,
 0.9150090638301467,
 0.9164523255574111,
 0.9104699239513777,
 0.9170207097082527,
 0.9147911115570295,
 0.8983633434179635,
 0.8982111198575603,
 0.9031985891795128,
 0.9092488405440492,
 0.9086609568233519,
 0.8960703655993213,
 0.8980287441900814,
 0.8939710157828432,
 0.8931248418049945,
 0.9255468702963244,
 0.9255468702963247,
 0.9278228244210703,
 0.9303333605962905,
 0.9325061575614121,
 0.9294254632521753,
 0.9252472971840199,
 0.88329942150570

In [26]:
# 计算策略指标
asset_end = netvalue_list[-1]
total_return = (asset_end-1)/1 # 总收益率
annual_return = pow((1+total_return),250/369)-1 # 年化收益率

print(total_return,annual_return)
# 0.32656080260972553 0.21101210079721788

0.3122031252267883 0.20211640214671833


In [27]:
# 日收益率列表
day_return = [(netvalue_list[i]-netvalue_list[i-1])/netvalue_list[i-1] for i in range(1,369)]
firstday_return = (netvalue_list[0]-1)/1 # 将第1个交易日的日收益率插入日收益率列表中
day_return.insert(0,firstday_return)

# day_return

In [28]:
# 夏普比率
import numpy as np
dayreturn_array = np.array(day_return) # list -> array
sigma = np.std(dayreturn_array, ddof=1) # np.std无偏样本标准差方式为 ddof = 1 # 波动率计算
annual_sigma = sigma * pow(250,0.5) # 年化标准差
sharpe = (annual_return-0.03)/annual_sigma

sharpe

1.1603989767159602

In [29]:
# 最大回撤
# 第 num 天的最大回撤值
def drawdown(dayreturn_lst, num):
    return max(dayreturn_lst[0:num]) - dayreturn_lst[-1] # 前 num 天的最大值 减去 当天 收益率

drawdown_list = [drawdown(day_return,i) for i in range(1,369)] # 368, 第1天没有回撤值
maxdrawdown = max(drawdown_list)

maxdrawdown

0.032040799750412234